# Fact Checker Application

## Step 1: Imports

In [1]:
import os

import numpy as np
from checkfake import procDataFake
from result import calcResult
from retrieve import genArticleData
from stance import procDataStance
from utils import (
    cleanString,
    delExisting,
    discardStanceArticle,
    discardFakeArticle,
    loadData,
    saveData,
    webRequest,
)

## Step 2: Base Setup

In [2]:
delExisting()
helper_path = os.path.dirname(os.path.dirname(os.getcwd()))

print("Base Completed")

Base Completed


## Step 3: Fact Data Retrieval and Saving

In [3]:
fact_query = input("Enter the fact that you wish to scrape articles for: ")
clean_query = cleanString(fact_query)

print(f"Cleaned Query String: {clean_query}")

Enter the fact that you wish to scrape articles for:  Was George Floyd Murdered?


Cleaned Query String: george floyd murdered


In [4]:
url = f"https://newsapi.org/v2/everything?q={clean_query}&apiKey="
json_dump = webRequest(url)

In [5]:
save_path = os.path.join(helper_path, "data/operational/query_result.json")
clean_article_data = genArticleData(json_dump["articles"])
print("Articles Retrieved from NewsAPI")

clean_article_data[0]

Articles Retrieved from NewsAPI


{'url': 'https://www.nytimes.com/2021/04/21/podcasts/the-daily/derek-chauvin-guilty-murder-george-floyd.html',
 'website': 'New York Times',
 'date': '2021-04-21T10:00:06Z',
 'title': 'Guilty of All Charges',
 'author': None,
 'text': 'Listen and follow The DailyApple Podcasts | Spotify | StitcherOn Tuesday, after three weeks of jury selection, another three weeks of testimony and 10 hours of deliberations, Derek Chauvin, a former Minneapolis police officer, was found guilty of murder in the death of George Floyd.The jurors found Mr. Chauvin guilty of all three charges: second-degree murder, third-degree murder and second-degree manslaughter. Sentencing will take place several weeks from now. Second-degree murder could mean as long as 40 years in prison.We look back on key moments from the trial and discuss the reactions to the guilty verdict.'}

In [6]:
saveData(clean_article_data, save_path)
print(f"Articles Saved at {save_path}")

Articles Saved at /home/apollo/Desktop/DJSCE/Projects/checkmate_fact_checker/data/operational/query_result.json


## Step 4: Stance Detection

In [7]:
main_df, model_df = loadData(save_path)

In [8]:
main_df.head(5)

,url,website,date,title,author,text
0,https://www.nytimes.com/2021/04/21/podcasts/th...,New York Times,2021-04-21 10:00:06+00:00,Guilty of All Charges,None,Listen and follow The DailyApple Podcasts | Sp...
1,https://www.nytimes.com/2021/04/20/us/darnella...,New York Times,2021-04-21 15:03:54+00:00,How Darnella Frazier's Video Upended The Minne...,Azi Paybarah,“May it please the court. Ladies and gentlemen...
2,https://www.nytimes.com/2021/04/21/us/politics...,New York Times,2021-04-21 22:45:04+00:00,What Might Bring Changes to Policing in Minnea...,Giovanni Russonello,Derek Chauvin had worked for almost two decade...
3,https://www.reuters.com/world/us/us-launch-pro...,Reuters,2021-04-21 15:25:00+00:00,US launches probe of Minneapolis police after ...,"Reuters,Sarah Lynch,Jan Wolfe","A view of the ""Say Their Names"" cemetery on th..."
4,https://www.cnn.com/2021/04/19/us/brooklyn-cen...,CNN,2021-04-19 21:11:46+00:00,Twin Cities are again the national flashpoint ...,"Nicole Chavez and Adrienne Broaddus, CNN","Brooklyn Center, Minnesota (CNN) A wooden Blac..."


In [9]:
model_df.head(5)

,text,title
0,Listen and follow The DailyApple Podcasts | Sp...,Guilty of All Charges
1,“May it please the court. Ladies and gentlemen...,How Darnella Frazier's Video Upended The Minne...
2,Derek Chauvin had worked for almost two decade...,What Might Bring Changes to Policing in Minnea...
3,"A view of the ""Say Their Names"" cemetery on th...",US launches probe of Minneapolis police after ...
4,"Brooklyn Center, Minnesota (CNN) A wooden Blac...",Twin Cities are again the national flashpoint ...


In [10]:
preds = procDataStance(model_df, helper_path)

agree = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[3]]
disagree = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[2]]
discuss = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[0]]
unrelated = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[1]]

print(f"Agree: \n{model_df['title'].iloc[agree]}\n")
print(f"Disgree: \n{model_df['title'].iloc[disagree]}\n")
print(f"Discuss: \n{model_df['title'].iloc[discuss]}\n")
print(f"Unrelated: \n{model_df['title'].iloc[unrelated]}\n")

print("Discarding articles with stance: Disagree and Unrelated")

Agree: 
2     What Might Bring Changes to Policing in Minnea...
3     US launches probe of Minneapolis police after ...
4     Twin Cities are again the national flashpoint ...
6         Biden and Harris on the Chauvin Trial Verdict
7     Tucker Carlson's reaction to Derek Chauvin's c...
8     'Justice' but still 'a long way to go': Reacti...
9     'Last Straw': Americans confront racism, viole...
10    George Floyd: US city on edge as jury delibera...
11    Floyd Brothers React To Verdict: 'This Is For ...
13    Prosecutor argues Chauvin’s ‘ego’ led to Floyd...
15    'Finally': America Reacts To Chauvin Guilty Ve...
16    El Salvador president says missing teen found,...
17    Celebration and tears: Reactions pour in after...
Name: title, dtype: object

Disgree: 
1     How Darnella Frazier's Video Upended The Minne...
14    NBA, MLB and NHL postpone Minneapolis games fo...
18    Where Efforts To Overhaul Policing Stand In Co...
Name: title, dtype: object

Discuss: 
0                   

In [11]:
model_df = discardStanceArticle(model_df, preds)
main_df = discardStanceArticle(main_df, preds)

main_df

,url,website,date,title,author,text
0,https://www.nytimes.com/2021/04/21/podcasts/th...,New York Times,2021-04-21 10:00:06+00:00,Guilty of All Charges,None,Listen and follow The DailyApple Podcasts | Sp...
1,https://www.nytimes.com/2021/04/21/us/politics...,New York Times,2021-04-21 22:45:04+00:00,What Might Bring Changes to Policing in Minnea...,Giovanni Russonello,Derek Chauvin had worked for almost two decade...
2,https://www.reuters.com/world/us/us-launch-pro...,Reuters,2021-04-21 15:25:00+00:00,US launches probe of Minneapolis police after ...,"Reuters,Sarah Lynch,Jan Wolfe","A view of the ""Say Their Names"" cemetery on th..."
3,https://www.cnn.com/2021/04/19/us/brooklyn-cen...,CNN,2021-04-19 21:11:46+00:00,Twin Cities are again the national flashpoint ...,"Nicole Chavez and Adrienne Broaddus, CNN","Brooklyn Center, Minnesota (CNN) A wooden Blac..."
4,https://boingboing.net/2021/04/21/merrick-garl...,Boing Boing,2021-04-21 14:50:38+00:00,Merrick Garland says Chauvin's guilty verdict ...,Carla Sinclair,On the day that Derek Chauvin murdered George ...
5,https://www.nytimes.com/2021/04/20/us/politics...,New York Times,2021-04-21 01:34:11+00:00,Biden and Harris on the Chauvin Trial Verdict,None,"For so many, it feels like it took all of that..."
6,https://www.businessinsider.com/tucker-carlson...,Business Insider,2021-04-21 10:46:26+00:00,Tucker Carlson's reaction to Derek Chauvin's c...,Mia Jankowicz,Tucker Carlson cast doubt on the motives of th...
7,https://www.reuters.com/business/legal/justice...,Reuters,2021-04-20 22:31:00+00:00,'Justice' but still 'a long way to go': Reacti...,Reuters,People react after the verdict in the trial of...
8,https://www.reuters.com/world/us/last-straw-am...,Reuters,2021-04-21 11:07:00+00:00,"'Last Straw': Americans confront racism, viole...",Brad Brooks,New York City mayoral candidate Dianne Morales...
9,https://www.bbc.com/news/world-us-canada-56806961,BBC News,2021-04-19 23:04:29+00:00,George Floyd: US city on edge as jury delibera...,https://www.facebook.com/bbcnews,George Floyd: US city on edge as jury delibera...


## Step 5: Fake News Detection

In [12]:
preds = procDataFake(model_df, helper_path)

true = [count for count, pred in enumerate(preds) if pred[0] > pred[1]]
fake = [count for count, pred in enumerate(preds) if pred[0] < pred[1]]

print(f"True: \n{model_df['title'].iloc[true]}\n")
print(f"Fake: \n{model_df['title'].iloc[fake]}\n")

print("Discarding Fake Articles")

True: 
0                                        guilty charges
1              might bring changes policing minneapolis
2     us launches probe minneapolis police chauvin c...
3         twin cities national flashpoint race policing
4     merrick garland says chauvins guilty verdict e...
5                    biden harris chauvin trial verdict
6     tucker carlsons reaction derek chauvins convic...
8     last straw americans confront racism violence ...
9     george floyd us city edge jury deliberates cha...
10           floyd brothers react verdict everyone held
11    prosecutor argues chauvin ’ ‘ ego ’ led floyd ...
12        finally america reacts chauvin guilty verdict
14    celebration tears reactions pour derek chauvin...
Name: title, dtype: object

Fake: 
7     justice still long way go reaction chauvin con...
13    el salvador president says missing teen found ...
Name: title, dtype: object

Discarding Fake Articles


In [13]:
model_df = discardFakeArticle(model_df, preds)
main_df = discardFakeArticle(main_df, preds)

main_df

,url,website,date,title,author,text
0,https://www.nytimes.com/2021/04/21/podcasts/th...,New York Times,2021-04-21 10:00:06+00:00,Guilty of All Charges,None,Listen and follow The DailyApple Podcasts | Sp...
1,https://www.nytimes.com/2021/04/21/us/politics...,New York Times,2021-04-21 22:45:04+00:00,What Might Bring Changes to Policing in Minnea...,Giovanni Russonello,Derek Chauvin had worked for almost two decade...
2,https://www.reuters.com/world/us/us-launch-pro...,Reuters,2021-04-21 15:25:00+00:00,US launches probe of Minneapolis police after ...,"Reuters,Sarah Lynch,Jan Wolfe","A view of the ""Say Their Names"" cemetery on th..."
3,https://www.cnn.com/2021/04/19/us/brooklyn-cen...,CNN,2021-04-19 21:11:46+00:00,Twin Cities are again the national flashpoint ...,"Nicole Chavez and Adrienne Broaddus, CNN","Brooklyn Center, Minnesota (CNN) A wooden Blac..."
4,https://boingboing.net/2021/04/21/merrick-garl...,Boing Boing,2021-04-21 14:50:38+00:00,Merrick Garland says Chauvin's guilty verdict ...,Carla Sinclair,On the day that Derek Chauvin murdered George ...
5,https://www.nytimes.com/2021/04/20/us/politics...,New York Times,2021-04-21 01:34:11+00:00,Biden and Harris on the Chauvin Trial Verdict,None,"For so many, it feels like it took all of that..."
6,https://www.businessinsider.com/tucker-carlson...,Business Insider,2021-04-21 10:46:26+00:00,Tucker Carlson's reaction to Derek Chauvin's c...,Mia Jankowicz,Tucker Carlson cast doubt on the motives of th...
7,https://www.reuters.com/world/us/last-straw-am...,Reuters,2021-04-21 11:07:00+00:00,"'Last Straw': Americans confront racism, viole...",Brad Brooks,New York City mayoral candidate Dianne Morales...
8,https://www.bbc.com/news/world-us-canada-56806961,BBC News,2021-04-19 23:04:29+00:00,George Floyd: US city on edge as jury delibera...,https://www.facebook.com/bbcnews,George Floyd: US city on edge as jury delibera...
9,https://www.npr.org/sections/trial-over-killin...,NPR,2021-04-21 00:28:35+00:00,Floyd Brothers React To Verdict: 'This Is For ...,Vanessa Romo,Floyd Brothers React To Verdict: 'This Is For ...


## Step 6: Stance Detection Similarity Matching

In [14]:
stance_df = model_df.loc[:, ["text"]]
stance_df["title"] = [fact_query] * len(stance_df.index)

stance_df

,text,title
0,listen follow dailyapple podcasts spotify stit...,Was George Floyd Murdered?
1,derek chauvin worked almost two decades minnea...,Was George Floyd Murdered?
2,view say names cemetery day verdict trial form...,Was George Floyd Murdered?
3,brooklyn center minnesota cnn wooden black pow...,Was George Floyd Murdered?
4,day derek chauvin murdered george floyd may 20...,Was George Floyd Murdered?
5,many feels like took judicial system deliver —...,Was George Floyd Murdered?
6,tucker carlson cast doubt motives jurors convi...,Was George Floyd Murdered?
7,new york city mayoral candidate dianne morales...,Was George Floyd Murdered?
8,george floyd us city edge jury deliberates cha...,Was George Floyd Murdered?
9,floyd brothers react verdict everyone held dow...,Was George Floyd Murdered?


In [15]:
preds = procDataStance(model_df, helper_path)

agree = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[3]]
disagree = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[2]]
discuss = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[0]]
unrelated = [count for count, pred in enumerate(preds) if np.amax(pred) == pred[1]]

print(f"Agree: \n{model_df['title'].iloc[agree]}\n")
print(f"Disgree: \n{model_df['title'].iloc[disagree]}\n")
print(f"Discuss: \n{model_df['title'].iloc[discuss]}\n")
print(f"Unrelated: \n{model_df['title'].iloc[unrelated]}\n")

Agree: 
1              might bring changes policing minneapolis
2     us launches probe minneapolis police chauvin c...
3         twin cities national flashpoint race policing
4     merrick garland says chauvins guilty verdict e...
6     tucker carlsons reaction derek chauvins convic...
7     last straw americans confront racism violence ...
8     george floyd us city edge jury deliberates cha...
9            floyd brothers react verdict everyone held
10    prosecutor argues chauvin ’ ‘ ego ’ led floyd ...
11        finally america reacts chauvin guilty verdict
Name: title, dtype: object

Disgree: 
0                                        guilty charges
12    celebration tears reactions pour derek chauvin...
Name: title, dtype: object

Discuss: 
5    biden harris chauvin trial verdict
Name: title, dtype: object

Unrelated: 
Series([], Name: title, dtype: object)



## Step 7: Final Results

In [16]:
conf_score, agree_idx, disagree_idx = calcResult(main_df, preds)

print(f"\nThe fact '{fact_query}' can be said to be true with {round(conf_score, 4)*100}% confidence.")


The fact 'Was George Floyd Murdered?' can be said to be true with 84.61999999999999% confidence.


In [17]:
agree_df = main_df.iloc[agree_idx]

save_path = os.path.join(helper_path, "data/operational/agree.json")
agree_df.to_json(save_path, orient="records", force_ascii=False, indent=4)

agree_df

,url,website,date,title,author,text
1,https://www.nytimes.com/2021/04/21/us/politics...,New York Times,2021-04-21 22:45:04+00:00,What Might Bring Changes to Policing in Minnea...,Giovanni Russonello,Derek Chauvin had worked for almost two decade...
2,https://www.reuters.com/world/us/us-launch-pro...,Reuters,2021-04-21 15:25:00+00:00,US launches probe of Minneapolis police after ...,"Reuters,Sarah Lynch,Jan Wolfe","A view of the ""Say Their Names"" cemetery on th..."
3,https://www.cnn.com/2021/04/19/us/brooklyn-cen...,CNN,2021-04-19 21:11:46+00:00,Twin Cities are again the national flashpoint ...,"Nicole Chavez and Adrienne Broaddus, CNN","Brooklyn Center, Minnesota (CNN) A wooden Blac..."
4,https://boingboing.net/2021/04/21/merrick-garl...,Boing Boing,2021-04-21 14:50:38+00:00,Merrick Garland says Chauvin's guilty verdict ...,Carla Sinclair,On the day that Derek Chauvin murdered George ...
5,https://www.nytimes.com/2021/04/20/us/politics...,New York Times,2021-04-21 01:34:11+00:00,Biden and Harris on the Chauvin Trial Verdict,None,"For so many, it feels like it took all of that..."
6,https://www.businessinsider.com/tucker-carlson...,Business Insider,2021-04-21 10:46:26+00:00,Tucker Carlson's reaction to Derek Chauvin's c...,Mia Jankowicz,Tucker Carlson cast doubt on the motives of th...
7,https://www.reuters.com/world/us/last-straw-am...,Reuters,2021-04-21 11:07:00+00:00,"'Last Straw': Americans confront racism, viole...",Brad Brooks,New York City mayoral candidate Dianne Morales...
8,https://www.bbc.com/news/world-us-canada-56806961,BBC News,2021-04-19 23:04:29+00:00,George Floyd: US city on edge as jury delibera...,https://www.facebook.com/bbcnews,George Floyd: US city on edge as jury delibera...
9,https://www.npr.org/sections/trial-over-killin...,NPR,2021-04-21 00:28:35+00:00,Floyd Brothers React To Verdict: 'This Is For ...,Vanessa Romo,Floyd Brothers React To Verdict: 'This Is For ...
10,https://www.washingtonpost.com/nation/2021/04/...,The Washington Post,2021-04-20 00:10:00+00:00,Prosecutor argues Chauvin’s ‘ego’ led to Floyd...,Holly Bailey,"The jury of 12 people — six White, four Black ..."


In [18]:
disagree_df = main_df.iloc[disagree_idx]

save_path = os.path.join(helper_path, "data/operational/disagree.json")
disagree_df.to_json(save_path, orient="records", force_ascii=False, indent=4)

disagree_df

,url,website,date,title,author,text
0,https://www.nytimes.com/2021/04/21/podcasts/th...,New York Times,2021-04-21 10:00:06+00:00,Guilty of All Charges,None,Listen and follow The DailyApple Podcasts | Sp...
12,https://news.yahoo.com/celebration-tears-react...,Yahoo Entertainment,2021-04-20 21:58:14+00:00,Celebration and tears: Reactions pour in after...,"Mitchell Willetts, Chacour Koop",The Daily BeastDavid Lee/NetflixThis is a prev...


In [19]:
print("Program Execution Complete. Please check agree.json and disagree.json for news articles")

Program Execution Complete. Please check agree.json and disagree.json for news articles
